# Tweets from bioRxiv and medRxiv

Find and display events from the CSHL DOI prefix 10.1101 from the last 7 days, and find the most tweeted preprints.

In [1]:
import sys
sys.path.append('../')
sys.path.insert(0, '..')

import pandas as pd # data analysis library
import json
import mrced2 # module to run event data queries
import os # some file manipulations
import math # some number manipulations
import altair.vegalite.v3 as alt # some data visualizations
from IPython.display import Markdown as md # some markdown manipulations
from datetime import datetime, date, timedelta # some date manipulations

In [2]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = (date.today() - timedelta(days = 7)).strftime('%Y-%m-%d')
end_date = date.today()

In [3]:
ed = mrced2.eventData(mailto = email)
ed.buildQuery({'obj-id.prefix' : prefix, 'source': 'twitter', 'rows': 500,'from-occurred-date' : start_date, 'until-occurred-date' : end_date})

https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&obj-id.prefix=10.1101&source=twitter&rows=500&from-occurred-date=2021-08-30&until-occurred-date=2021-09-06


In [4]:
ed.runQuery(retry = 5)

Event Data query started...
API query complete  200
output file written to 1101/tweets.json


In [5]:
pages = math.ceil(ed.events.getHits() / 1000)

12636 events found


In [6]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = date.today() - timedelta(days = 7)
end_date = date.today()

# find the all result pages for the search
ed = mrced2.eventData(mailto = email)
ed.getAllPages(pages, {'rows': 1000, 'obj-id.prefix' : prefix, 'source': 'twitter', 'from-occurred-date' : start_date, 'until-occurred-date' : end_date}, fileprefix = '1101/tweets_') 

https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-08-30&until-occurred-date=2021-09-06
Event Data query started...
API query complete  200
output file written to 1101/tweets_0000.json
https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&cursor=0633d756-1645-4176-8937-68fd63f8a7d6&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-08-30&until-occurred-date=2021-09-06
Event Data query started...
API query complete  200
output file written to 1101/tweets_0001.json
https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&cursor=d2e8a6a1-394c-424e-93bc-9704d9a42d01&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-08-30&until-occurred-date=2021-09-06
Event Data query started...
API query complete  200
output file written to 1101/tweets_0002.json
https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&curso

Initialisation to look at the properties of the results.

In [7]:
jd1 = mrced2.eventRecord() # instance of a class to interpret the events
files = os.listdir('1101') # get all the filenames

jd1.mergeJsons(files, folder = '1101') # load the json event data from multiple files

failed to load preprint_tweets_2021-07-29.csv
failed to load .DS_Store
failed to load .gitkeep
failed to load preprint_tweets_2021-08-30.csv
failed to load preprint_tweets_2021-08-09.csv
failed to load preprint_tweets_2021-08-23.csv
failed to load preprint_tweets_2021-08-02.csv
failed to load preprint_tweets_2021-08-16.csv
failed to load .ipynb_checkpoints
failed to load preprint_tweets_2021-07-26.csv
output file written to 1101/tweets.json


In [8]:
js = json.load(open("1101/tweets.json"))
df = pd.json_normalize(js, record_path = ['message', 'events'])
gdf = df.groupby(['obj_id']).size().reset_index(name='count').sort_values('count', ascending=False)
cdf = gdf[gdf['count'] >= 3]

In [9]:
email = "info@front-matter.io"
rest = mrced2.restApi(email = email)

data = []
for index, row in cdf.iterrows():
    rest.runQuery(row)
    if rest.work is not None and date.fromisoformat(rest.work["posted"]) >= start_date:
        data.append(rest.work)
print(data)
    
tdf = pd.DataFrame(data, columns=['doi','tweets','archive','subject-area','covid','title','authors','abstract','posted'])
tdf.to_csv('1101/preprint_tweets_' + str(date.today().strftime('%Y-%m-%d')) + '.csv')

tdf.head(50)

REST API query started for 10.1101/2021.07.08.21260210...
REST API query complete  200
REST API query started for 10.1101/2021.07.13.452194...
REST API query complete  200
REST API query started for 10.1101/2021.07.31.21261387...
REST API query complete  200
REST API query started for 10.1101/2021.04.19.21255739...
REST API query complete  200
REST API query started for 10.1101/2021.07.06.21259982...
REST API query complete  200
REST API query started for 10.1101/2021.05.06.21256755...
REST API query complete  200
REST API query started for 10.1101/2021.06.01.21258176...
REST API query complete  200
REST API query started for 10.1101/2021.08.27.21262741...
REST API query complete  200
REST API query started for 10.1101/2021.08.08.21261768...
REST API query complete  200
REST API query started for 10.1101/2021.07.28.21261295...
REST API query complete  200
REST API query started for 10.1101/2021.08.19.456982...
REST API query complete  200
REST API query started for 10.1101/2021.08.15.2

,doi,tweets,archive,subject-area,covid,title,authors,abstract,posted
0,10.1101/2021.08.31.458411,34,bioRxiv,Plant Biology,False,An Arabidopsis root phloem pole cell atlas rev...,"[{'name': 'Sofia Otero'}, {'name': 'Iris Sevil...",<p>Single cell sequencing has recently allowed...,2021-09-01
1,10.1101/2021.08.31.21262943,29,medRxiv,Public and Global Health,True,Ensemble forecast of COVID-19 in Karnataka for...,"[{'name': 'Sashikumaar Ganesan'}, {'name': 'De...",<p>We present an ensemble forecast for Wave-3 ...,2021-09-02
2,10.1101/2021.09.03.458877,27,bioRxiv,Pharmacology and Toxicology,True,Discovery of highly potent pancoronavirus fusi...,"[{'name': 'Francesca Curreli'}, {'name': 'Shah...",<p>We report the discovery of a series of benz...,2021-09-03
3,10.1101/2021.08.30.458146,24,bioRxiv,Developmental Biology,False,The control of transcriptional memory by stabl...,"[{'name': 'Maëlle Bellec'}, {'name': 'Jérémy D...",<p>To maintain cellular identities during deve...,2021-08-30
4,10.1101/2021.08.31.458428,9,bioRxiv,Cell Biology,False,Characterisation of a nucleo-adhesome,"[{'name': 'Adam Byron'}, {'name': 'Billie G. C...",<p>In addition to central functions in cell ad...,2021-08-31
5,10.1101/2021.08.29.458102,8,bioRxiv,Molecular Biology,False,Exo1-protected DNA nicks direct crossover form...,"[{'name': 'Michael Gioia'}, {'name': 'Lisette ...",<p>In most sexually reproducing organisms cros...,2021-08-30
6,10.1101/2021.09.01.458557,7,bioRxiv,Cell Biology,False,Functional fluorescently-tagged human profilin,"[{'name': 'Morgan L. Pimm'}, {'name': 'Xinbei ...",<p>Profilin is an essential regulator of actin...,2021-09-02
7,10.1101/2021.08.30.458230,7,bioRxiv,Neuroscience,False,Disentangling the flow of signals between popu...,"[{'name': 'Evren Gokcen'}, {'name': 'Anna I. J...",<p>Technological advances have granted the abi...,2021-09-01
8,10.1101/2021.08.30.458226,7,bioRxiv,Bioinformatics,False,De Novo Prediction of RNA 3D Structures with D...,"[{'name': 'Julius Ramakers'}, {'name': 'Christ...",<p>We present a Deep Learning approach to pred...,2021-09-01
9,10.1101/2021.08.29.21262789,7,medRxiv,Epidemiology,True,Vaccines alone are no silver bullets: a modeli...,"[{'name': 'Dhesi Baha Raja'}, {'name': 'Nur As...",<sec><title>Background</title><p>The computer ...,2021-08-30


### Tweets of bioRxiv and medRxiv preprints

In [10]:
num_rows = tdf['archive'].count()
num_covid = tdf['covid'].value_counts(ascending=True)[1]
num_biorxiv = tdf['archive'].value_counts(ascending=True)[1]
num_medrxiv = tdf['archive'].value_counts(ascending=True)[0]
end_date = date.today().strftime('%Y-%m-%d')
max_count = tdf['tweets'].max()

md('{} preprints (including {} covering SARS-CoV-2, {} from bioRxiv and {} from medRxiv) published in the last 7 days before {} had been tweeted at least 3 times (maximum {}).'.format(num_rows, num_covid, num_biorxiv, num_medrxiv, end_date, max_count))

29 preprints (including 6 covering SARS-CoV-2, 25 from bioRxiv and 4 from medRxiv) published in the last 7 days before 2021-09-06 had been tweeted at least 3 times (maximum 34).